In [2]:
# This part is to get FB user access tokens.

import requests

app_id = '1048765005506273'
app_secret = '5622f0056c5222df060cb74aff8d4f04'
user_short_token = 'EAAYCDwuzUQ4BACMMJAZA4WuGZBxVnEwOguqBuwM1NfyCWGHeZBZBvwD2iKckHICBVtWdu3JZBqTYm16174fYnIyaTW2ZAfLXwwvmUZBXN16kIAvSiTIRymAei8ETWZCDAlzIcZAX5v4iZBnbW9azt5SHFBsbNL9qIMXzXuFrdD2FGuEXjjyR4YPXwRSumWa8YMl44GVHKFyJPBxwZDZD'
access_token_url = "https://graph.facebook.com/oauth/access_token?grant_type=fb_exchange_token&client_id={}&client_secret={}&fb_exchange_token={}".format(app_id, app_secret, user_short_token)

r = requests.get(access_token_url)

access_token_info = r.json()

user_long_token = access_token_info['access_token']

print(user_long_token)

KeyError: 'access_token'

In [6]:
# Currently no function

import facebook
graph = facebook.GraphAPI(access_token = user_long_token)
pages_data = graph.get_object("/me/accounts")
print(pages_data)

ModuleNotFoundError: No module named 'facebook'

In [3]:
# This part is to fetch Feed data from Facebook Graph API and store it to feed_data.json and feed_data.csv

import json
import csv
from urllib.request import urlopen

user_short_token = 'EAAYCDwuzUQ4BACMMJAZA4WuGZBxVnEwOguqBuwM1NfyCWGHeZBZBvwD2iKckHICBVtWdu3JZBqTYm16174fYnIyaTW2ZAfLXwwvmUZBXN16kIAvSiTIRymAei8ETWZCDAlzIcZAX5v4iZBnbW9azt5SHFBsbNL9qIMXzXuFrdD2FGuEXjjyR4YPXwRSumWa8YMl44GVHKFyJPBxwZDZD'
with urlopen(
 "https://graph.facebook.com/v6.0/me?fields=id%2Cname%2Cfeed%7Bcreated_time%2Cdescription%7D%2Cposts%7Bcreated_time%2Cmessage%2Cdescription%2Ccaption%7D&access_token={}".format(user_short_token)) as response:
    source = response.read()
data = json.loads(source)

some_data = data['feed']

outfile = open("feed_data.csv", "w")

fieldNames = ['created_time', 'id', 'description']

writer = csv.DictWriter(outfile, fieldnames = fieldNames)
writer.writeheader()

while len(some_data['data']) != 0:
    with open('feed_data.json', 'a+', encoding='utf-8') as f:
        json.dump(some_data['data'], f, ensure_ascii=False, indent=4)
    
    for row in some_data['data']:
        writer.writerow(row)
#         print(row)
    
    paging_next = some_data['paging']['next']
    
    with urlopen(paging_next) as response:
        source = response.read()
    some_data = json.loads(source)


In [14]:
# To get the final csv file to implement moneylearn on it


import numpy as np
import pandas as pd


df = pd.read_csv("feed_data.csv")
df = df[pd.notnull(df['description'])]
df['Response'] = np.nan
df.reset_index(drop=True, inplace=True)
df.to_csv("feed_clean.csv")#, index = False)

In [32]:
# calling monkeylearn api


import numpy as np
import pandas as pd
from monkeylearn import MonkeyLearn

senti_value = pd.Series([])
confidence = pd.Series([])

ml = MonkeyLearn('4f7ce703b1f1ee821b1cf4eaf34dbf9c748df062')
df = pd.read_csv("final.csv")
model_id = 'cl_pi3C7JiL'

for i in range(len(df)):  # Change head() 
    temp = []
    column = df['description'][i]
    temp.append(column)
    result = ml.classifiers.classify(model_id, temp)
    senti_value[i] = result.body[0]['classifications'][0]['tag_name']
    confidence[i] = result.body[0]['classifications'][0]['confidence']  
    print(result.body[0]['classifications'])

[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.496}]
[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.445}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.758}]
[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.59}]
[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.864}]
[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.811}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.737}]
[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.985}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.67}]
[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.505}]
[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.516}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.802}]
[{'tag_name': 'Negative', 'tag_id': 60333049, 'confidence': 0.434}]
[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.988}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confiden

In [33]:
# Inserting Senti_Value and Confidence column

df.insert(4, "Senti_Value", senti_value)
df.insert(5, "Confidence", confidence)
df.head()

,Unnamed: 0,created_time,id,description,Senti_Value,Confidence,Response
0,0,2019-11-08T19:14:55+0000,2718139118302363_2524814404301503,Two years ago at the Astros World Series parad...,Positive,0.496,NaN
1,1,2019-09-30T16:50:10+0000,2718139118302363_2444947955621482,"This beluga is the purest, most magical creatu...",Positive,0.445,NaN
2,2,2019-01-03T19:18:41+0000,2718139118302363_1994092734040342,Snowy Myeongnyundang,Neutral,0.758,NaN
3,3,2018-12-01T18:30:25+0000,2718139118302363_1934358063347143,It's so fluffy I'm gonna die! 😍,Positive,0.590,NaN
4,4,2018-11-30T14:17:33+0000,2718139118302363_1932679396848343,Spa day fun day!\n\nby Dean The Basset,Positive,0.864,NaN


In [34]:
# Do not Execute

df.drop(columns = ['Response'], inplace = True)
df.head()

,Unnamed: 0,created_time,id,description,Senti_Value,Confidence
0,0,2019-11-08T19:14:55+0000,2718139118302363_2524814404301503,Two years ago at the Astros World Series parad...,Positive,0.496
1,1,2019-09-30T16:50:10+0000,2718139118302363_2444947955621482,"This beluga is the purest, most magical creatu...",Positive,0.445
2,2,2019-01-03T19:18:41+0000,2718139118302363_1994092734040342,Snowy Myeongnyundang,Neutral,0.758
3,3,2018-12-01T18:30:25+0000,2718139118302363_1934358063347143,It's so fluffy I'm gonna die! 😍,Positive,0.590
4,4,2018-11-30T14:17:33+0000,2718139118302363_1932679396848343,Spa day fun day!\n\nby Dean The Basset,Positive,0.864


In [35]:
# Grouping the Senti_Value

df['Senti_Value'] = df['Senti_Value'].replace({ 'Positive' : 1, 'Neutral' : 0, 'Negative' : -1 })

In [36]:
df.head()

,Unnamed: 0,created_time,id,description,Senti_Value,Confidence
0,0,2019-11-08T19:14:55+0000,2718139118302363_2524814404301503,Two years ago at the Astros World Series parad...,1,0.496
1,1,2019-09-30T16:50:10+0000,2718139118302363_2444947955621482,"This beluga is the purest, most magical creatu...",1,0.445
2,2,2019-01-03T19:18:41+0000,2718139118302363_1994092734040342,Snowy Myeongnyundang,0,0.758
3,3,2018-12-01T18:30:25+0000,2718139118302363_1934358063347143,It's so fluffy I'm gonna die! 😍,1,0.590
4,4,2018-11-30T14:17:33+0000,2718139118302363_1932679396848343,Spa day fun day!\n\nby Dean The Basset,1,0.864


In [57]:
# Assigning weightage to DataTime

total_rows = len(df.index)
# total_rows = 21
even = False

time_weigh = pd.Series([])

if total_rows % 2 == 0:
    even = True

if even == True:
    i = total_rows/2 - 1
    time_weigh[i] = 25
    i = i-1
    while(i >= 0):
        time_weigh[i] = time_weigh[i+1]/2
        i = i-1
        
    i = total_rows/2
    time_weigh[i] = 25
    i = i+1
    while(i < total_rows):
        time_weigh[i] = time_weigh[i-1]/2
        i = i+1
else:
    i = total_rows/2 - 0.5
    time_weigh[i] = 50/3
    time_weigh[i+1] = 50/3
    time_weigh[i-1] = 50/3
    time_weigh[i-2] = 25/2
    i = i - 3
    while (i >= 0):
        time_weigh[i] = time_weigh[i+1]/2
        i = i - 1
        
    i = total_rows/2 - 0.5
    time_weigh[i+2] = 25/2
    i = i+3
    while (i < total_rows):
        time_weigh[i] = time_weigh[i-1]/2
        i = i+1
        
# print(time_weigh)

In [58]:
# Inserting Time_Weight

df.insert(2, "Time_Weight", time_weigh)
df.head()

,Unnamed: 0,created_time,Time_Weight,id,description,Senti_Value,Confidence
0,0,2019-11-08T19:14:55+0000,2.273737e-11,2718139118302363_2524814404301503,Two years ago at the Astros World Series parad...,1,0.496
1,1,2019-09-30T16:50:10+0000,4.547474e-11,2718139118302363_2444947955621482,"This beluga is the purest, most magical creatu...",1,0.445
2,2,2019-01-03T19:18:41+0000,9.094947e-11,2718139118302363_1994092734040342,Snowy Myeongnyundang,0,0.758
3,3,2018-12-01T18:30:25+0000,1.818989e-10,2718139118302363_1934358063347143,It's so fluffy I'm gonna die! 😍,1,0.590
4,4,2018-11-30T14:17:33+0000,3.637979e-10,2718139118302363_1932679396848343,Spa day fun day!\n\nby Dean The Basset,1,0.864


In [90]:
# Do not Execute

# df.drop(columns = ['Confi_Weight'], inplace = True)
# df.head()

In [84]:
# Ranking based on Confidence

df['Confi_Rank'] = df['Confidence'].rank(ascending = 0, method = 'dense')
# df.sort_values('Confi_Rank', inplace = True)
df = df.set_index('Confi_Rank')
df = df.sort_index()
df.head()

,Unnamed: 0,created_time,Time_Weight,id,description,Senti_Value,Confidence
Confi_Rank,,,,,,,
1.0,13,2018-10-12T10:58:21+0000,1.862645e-07,2718139118302363_1863695673746716,"Here it is, Usain Bolt, the footballer, scores...",1,0.988
2.0,7,2018-10-21T07:34:59+0000,2.910383e-09,2718139118302363_1875687702547513,Always accurate\n\nInstagram : Instagram.com/S...,1,0.985
3.0,24,2018-08-15T16:41:54+0000,3.814697e-04,2718139118302363_1786267578156193,classy,1,0.980
4.0,57,2016-05-11T06:16:20+0000,3.814697e-04,2718139118302363_965660893550203,A police dog who played a major role in the 26...,1,0.964
5.0,18,2018-09-08T03:26:16+0000,5.960464e-06,2718139118302363_1820480931401524,This video perfectly captures the feeling that...,1,0.951


In [85]:
df

,Unnamed: 0,created_time,Time_Weight,id,description,Senti_Value,Confidence
Confi_Rank,,,,,,,
1.0,13,2018-10-12T10:58:21+0000,1.862645e-07,2718139118302363_1863695673746716,"Here it is, Usain Bolt, the footballer, scores...",1,0.988
2.0,7,2018-10-21T07:34:59+0000,2.910383e-09,2718139118302363_1875687702547513,Always accurate\n\nInstagram : Instagram.com/S...,1,0.985
3.0,24,2018-08-15T16:41:54+0000,3.814697e-04,2718139118302363_1786267578156193,classy,1,0.980
4.0,57,2016-05-11T06:16:20+0000,3.814697e-04,2718139118302363_965660893550203,A police dog who played a major role in the 26...,1,0.964
5.0,18,2018-09-08T03:26:16+0000,5.960464e-06,2718139118302363_1820480931401524,This video perfectly captures the feeling that...,1,0.951
6.0,32,2018-07-01T08:45:25+0000,9.765625e-02,2718139118302363_1714057288710556,#MemeTrends #June2018,0,0.933
7.0,78,2015-03-26T16:21:55+0000,1.818989e-10,2718139118302363_750908021692159,True love :P,1,0.925
8.0,77,2015-04-18T04:44:36+0000,3.637979e-10,2718139118302363_762566567192971,Share this with the world to let them know you...,1,0.922
9.0,68,2015-12-30T11:15:22+0000,1.862645e-07,2718139118302363_885088401607453,http://www.youtube.com/subscription_center?add...,0,0.921


In [87]:
# Assigning weightage based on Confidence

confi_weigh = pd.Series([])

confi_weigh[1] = 50

for ind in df.index:
    if (ind != 1.0):
        confi_weigh[ind] = confi_weigh[ind-1]/2
#     print(ind)

# print(confi_weigh)

In [91]:
# Inserting Confidence Weight

df.insert(7, "Confi_Weight", confi_weigh)
df

,Unnamed: 0,created_time,Time_Weight,id,description,Senti_Value,Confidence,Confi_Weight
Confi_Rank,,,,,,,,
1.0,13,2018-10-12T10:58:21+0000,1.862645e-07,2718139118302363_1863695673746716,"Here it is, Usain Bolt, the footballer, scores...",1,0.988,5.000000e+01
2.0,7,2018-10-21T07:34:59+0000,2.910383e-09,2718139118302363_1875687702547513,Always accurate\n\nInstagram : Instagram.com/S...,1,0.985,2.500000e+01
3.0,24,2018-08-15T16:41:54+0000,3.814697e-04,2718139118302363_1786267578156193,classy,1,0.980,1.250000e+01
4.0,57,2016-05-11T06:16:20+0000,3.814697e-04,2718139118302363_965660893550203,A police dog who played a major role in the 26...,1,0.964,6.250000e+00
5.0,18,2018-09-08T03:26:16+0000,5.960464e-06,2718139118302363_1820480931401524,This video perfectly captures the feeling that...,1,0.951,3.125000e+00
6.0,32,2018-07-01T08:45:25+0000,9.765625e-02,2718139118302363_1714057288710556,#MemeTrends #June2018,0,0.933,1.562500e+00
7.0,78,2015-03-26T16:21:55+0000,1.818989e-10,2718139118302363_750908021692159,True love :P,1,0.925,7.812500e-01
8.0,77,2015-04-18T04:44:36+0000,3.637979e-10,2718139118302363_762566567192971,Share this with the world to let them know you...,1,0.922,3.906250e-01
9.0,68,2015-12-30T11:15:22+0000,1.862645e-07,2718139118302363_885088401607453,http://www.youtube.com/subscription_center?add...,0,0.921,1.953125e-01


In [92]:
# Accumulating overall score for each Feed post


df['Overall'] = df['Time_Weight'] * df['Senti_Value'] * df['Confi_Weight']

In [96]:
# Saving the final table

df.to_csv('feed_complete.csv')

In [95]:
# Final score based on Feed data

feed_score = df['Overall'].sum()
print(feed_score)

-0.009507702558322575


In [109]:
# Fetch Post data from Facebook Graph API and store it to post_data.json and post_data.csv

import json
import csv
from urllib.request import urlopen

with urlopen(
 "https://graph.facebook.com/v6.0/me?fields=id%2Cname%2Cposts%7Bmessage%2Ccreated_time%7D&access_token={}".format(user_long_token)) as response:
    source = response.read()
    
data = json.loads(source)

some_data = data['posts']

outfile = open("post_data.csv", "w")

fieldNames = ['created_time', 'id', 'message']

writer = csv.DictWriter(outfile, fieldnames = fieldNames)
writer.writeheader()

while len(some_data['data']) != 0:
    with open('post_data.json', 'a+', encoding='utf-8') as f:
        json.dump(some_data['data'], f, ensure_ascii=False, indent=4)
    
    for row in some_data['data']:
        writer.writerow(row)
#         print(row)
    
    paging_next = some_data['paging']['next']
    
    with urlopen(paging_next) as response:
        source = response.read()
    some_data = json.loads(source)


In [120]:
import numpy as np
import pandas as pd


df = pd.read_csv("post_data.csv")

df = df[pd.notnull(df['message'])]

df.reset_index(drop = True, inplace=True)
df.to_csv("post_clean.csv")#, index = False

In [121]:
df.head()

,created_time,id,message
0,2020-01-05T15:20:46+0000,2718139118302363_2653380884778187,🌻
1,2019-12-30T18:51:10+0000,2718139118302363_2640213602761582,GG
2,2019-11-11T07:29:50+0000,2718139118302363_2530213200428290,Sir
3,2019-09-11T17:49:24+0000,2718139118302363_2409664199149858,To good life and good people. 🍻
4,2019-08-18T08:48:21+0000,2718139118302363_2365157633600515,Sugar Daddy. XDDDDDD


In [122]:
# calling monkeylearn api


import numpy as np
import pandas as pd
from monkeylearn import MonkeyLearn

senti_value = pd.Series([])
confidence = pd.Series([])

ml = MonkeyLearn('4f7ce703b1f1ee821b1cf4eaf34dbf9c748df062')
df = pd.read_csv("post_clean.csv")
model_id = 'cl_pi3C7JiL'


for i in range(len(df)):  # Change head() 
    temp = []
    column = df['message'][i]
    temp.append(column)
    result = ml.classifiers.classify(model_id, temp)
    
    senti_value[i] = result.body[0]['classifications'][0]['tag_name']
    confidence[i] = result.body[0]['classifications'][0]['confidence']
    
    print(result.body[0]['classifications'])


[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.575}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.829}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.747}]
[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.685}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.552}]
[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.927}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.485}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.521}]
[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.94}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.795}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.63}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.809}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.672}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.466}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0

[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.993}]
[{'tag_name': 'Negative', 'tag_id': 60333049, 'confidence': 0.683}]
[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.974}]
[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.677}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.549}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.844}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.791}]
[{'tag_name': 'Negative', 'tag_id': 60333049, 'confidence': 0.906}]
[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.602}]
[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.973}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.616}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.634}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.766}]
[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.851}]
[{'tag_name': 'Negative', 'tag_id': 60333049, 'confide

PlanQueryLimitError: Error PLAN_QUERY_LIMIT: Request Throttled. You do not have enough queries left to perform this action. 1 queries are required, but you have 0 queries remaining.

In [123]:
print(senti_value)

0      Positive
1       Neutral
2       Neutral
3      Positive
4       Neutral
5      Positive
6       Neutral
7       Neutral
8      Positive
9       Neutral
10      Neutral
11      Neutral
12      Neutral
13      Neutral
14      Neutral
15      Neutral
16      Neutral
17      Neutral
18     Negative
19      Neutral
20      Neutral
21      Neutral
22     Positive
23      Neutral
24     Positive
25     Positive
26      Neutral
27      Neutral
28      Neutral
29      Neutral
         ...   
153    Positive
154    Positive
155     Neutral
156    Negative
157    Positive
158    Negative
159    Positive
160    Positive
161    Positive
162     Neutral
163     Neutral
164    Positive
165    Positive
166    Positive
167    Positive
168     Neutral
169    Negative
170    Positive
171    Positive
172    Positive
173     Neutral
174    Positive
175     Neutral
176     Neutral
177    Positive
178    Negative
179     Neutral
180    Positive
181    Positive
182    Positive
Length: 183, dtype: obje

In [124]:
ml = MonkeyLearn('6e5e8ffc14f72febc9fd671de79293015adfac51')
df = pd.read_csv("post_clean.csv")
model_id = 'cl_pi3C7JiL'


for i in range(183, len(df)):  # Change head() 
    temp = []
    column = df['message'][i]
    temp.append(column)
    result = ml.classifiers.classify(model_id, temp)
    
    senti_value[i] = result.body[0]['classifications'][0]['tag_name']
    confidence[i] = result.body[0]['classifications'][0]['confidence']
    
    print(result.body[0]['classifications'])


[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.616}]
[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.792}]
[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.726}]
[{'tag_name': 'Negative', 'tag_id': 60333049, 'confidence': 0.52}]
[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.985}]
[{'tag_name': 'Negative', 'tag_id': 60333049, 'confidence': 0.875}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.546}]
[{'tag_name': 'Negative', 'tag_id': 60333049, 'confidence': 0.523}]
[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.899}]
[{'tag_name': 'Negative', 'tag_id': 60333049, 'confidence': 0.523}]
[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.812}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.46}]
[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.818}]
[{'tag_name': 'Negative', 'tag_id': 60333049, 'confidence': 0.826}]
[{'tag_name': 'Positive', 'tag_id': 60333048, 'confid

[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.5}]
[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.615}]
[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.587}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.381}]
[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.881}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.831}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.44}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.642}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.801}]
[{'tag_name': 'Negative', 'tag_id': 60333049, 'confidence': 0.392}]
[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence': 0.983}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.733}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.764}]
[{'tag_name': 'Neutral', 'tag_id': 60333050, 'confidence': 0.543}]
[{'tag_name': 'Positive', 'tag_id': 60333048, 'confidence':

In [125]:
# Inserting Senti_Value and Confidence column

df.insert(4, "Senti_Value", senti_value)
df.insert(5, "Confidence", confidence)
df.head()

,Unnamed: 0,created_time,id,message,Senti_Value,Confidence
0,0,2020-01-05T15:20:46+0000,2718139118302363_2653380884778187,🌻,Positive,0.575
1,1,2019-12-30T18:51:10+0000,2718139118302363_2640213602761582,GG,Neutral,0.829
2,2,2019-11-11T07:29:50+0000,2718139118302363_2530213200428290,Sir,Neutral,0.747
3,3,2019-09-11T17:49:24+0000,2718139118302363_2409664199149858,To good life and good people. 🍻,Positive,0.685
4,4,2019-08-18T08:48:21+0000,2718139118302363_2365157633600515,Sugar Daddy. XDDDDDD,Neutral,0.552


In [126]:
# Grouping the Senti_Value

df['Senti_Value'] = df['Senti_Value'].replace({ 'Positive' : 1, 'Neutral' : 0, 'Negative' : -1 })

In [127]:
# Assigning weightage to DataTime

total_rows = len(df.index)
# total_rows = 21
even = False

time_weigh = pd.Series([])

if total_rows % 2 == 0:
    even = True

if even == True:
    i = total_rows/2 - 1
    time_weigh[i] = 25
    i = i-1
    while(i >= 0):
        time_weigh[i] = time_weigh[i+1]/2
        i = i-1
        
    i = total_rows/2
    time_weigh[i] = 25
    i = i+1
    while(i < total_rows):
        time_weigh[i] = time_weigh[i-1]/2
        i = i+1
else:
    i = total_rows/2 - 0.5
    time_weigh[i] = 50/3
    time_weigh[i+1] = 50/3
    time_weigh[i-1] = 50/3
    time_weigh[i-2] = 25/2
    i = i - 3
    while (i >= 0):
        time_weigh[i] = time_weigh[i+1]/2
        i = i - 1
        
    i = total_rows/2 - 0.5
    time_weigh[i+2] = 25/2
    i = i+3
    while (i < total_rows):
        time_weigh[i] = time_weigh[i-1]/2
        i = i+1
        
# print(time_weigh)

In [128]:
# Inserting Time_Weight

df.insert(2, "Time_Weight", time_weigh)
df.head()

,Unnamed: 0,created_time,Time_Weight,id,message,Senti_Value,Confidence
0,0,2020-01-05T15:20:46+0000,6.372368e-56,2718139118302363_2653380884778187,🌻,1,0.575
1,1,2019-12-30T18:51:10+0000,1.274474e-55,2718139118302363_2640213602761582,GG,0,0.829
2,2,2019-11-11T07:29:50+0000,2.548947e-55,2718139118302363_2530213200428290,Sir,0,0.747
3,3,2019-09-11T17:49:24+0000,5.097894e-55,2718139118302363_2409664199149858,To good life and good people. 🍻,1,0.685
4,4,2019-08-18T08:48:21+0000,1.019579e-54,2718139118302363_2365157633600515,Sugar Daddy. XDDDDDD,0,0.552


In [129]:
# Ranking based on Confidence

df['Confi_Rank'] = df['Confidence'].rank(ascending = 0, method = 'dense')
# df.sort_values('Confi_Rank', inplace = True)
df = df.set_index('Confi_Rank')
df = df.sort_index()
df.head()

,Unnamed: 0,created_time,Time_Weight,id,message,Senti_Value,Confidence
Confi_Rank,,,,,,,
1.0,108,2016-03-23T08:49:54+0000,2.067952e-23,2718139118302363_931239206992372,Happy Annual Groping Festival! :D ;)\nEnjoy an...,1,0.998
2.0,254,2014-09-26T03:39:46+0000,1.355253e-18,2718139118302363_647905248659104,awesome...must watch,1,0.997
2.0,93,2016-04-09T20:26:59+0000,6.310887e-28,2718139118302363_946425495473743,Now I came to know from where the Bollywood mo...,1,0.997
2.0,238,2014-12-25T07:00:11+0000,8.881784e-14,2718139118302363_697666540349641,It's been a great year! Thanks for being a par...,1,0.997
2.0,146,2015-12-26T20:01:55+0000,5.684342e-12,2718139118302363_883427491773544,My all time favorite people! :),1,0.997


In [130]:
# Assigning weightage based on Confidence

confi_weigh = pd.Series([])

confi_weigh[1] = 50

for ind in df.index:
    if (ind != 1.0):
        confi_weigh[ind] = confi_weigh[ind-1]/2
#     print(ind)

# print(confi_weigh)

In [131]:
# Inserting Confidence Weight

df.insert(7, "Confi_Weight", confi_weigh)
df

,Unnamed: 0,created_time,Time_Weight,id,message,Senti_Value,Confidence,Confi_Weight
Confi_Rank,,,,,,,,
1.0,108,2016-03-23T08:49:54+0000,2.067952e-23,2718139118302363_931239206992372,Happy Annual Groping Festival! :D ;)\nEnjoy an...,1,0.998,5.000000e+01
2.0,254,2014-09-26T03:39:46+0000,1.355253e-18,2718139118302363_647905248659104,awesome...must watch,1,0.997,2.500000e+01
2.0,93,2016-04-09T20:26:59+0000,6.310887e-28,2718139118302363_946425495473743,Now I came to know from where the Bollywood mo...,1,0.997,2.500000e+01
2.0,238,2014-12-25T07:00:11+0000,8.881784e-14,2718139118302363_697666540349641,It's been a great year! Thanks for being a par...,1,0.997,2.500000e+01
2.0,146,2015-12-26T20:01:55+0000,5.684342e-12,2718139118302363_883427491773544,My all time favorite people! :),1,0.997,2.500000e+01
3.0,68,2016-08-07T16:08:23+0000,1.880791e-35,2718139118302363_1025653060884319,It never fails to impress you. :),1,0.996,1.250000e+01
3.0,213,2015-03-04T22:26:44+0000,2.980232e-06,2718139118302363_739381759511452,"Excellent story, awesome Casting...!!",1,0.996,1.250000e+01
4.0,161,2015-11-11T02:19:05+0000,1.862645e-07,2718139118302363_862160223900271,Happy Diwali Folks.. :)\nGo Green & hv a blast...,1,0.995,6.250000e+00
5.0,148,2015-12-24T16:58:57+0000,2.273737e-11,2718139118302363_881818121934481,Merry Christmas Folks!! :D Ho Ho Ho !! :D :D,1,0.994,3.125000e+00


In [132]:
# Accumulating overall score for each Feed post


df['Overall'] = df['Time_Weight'] * df['Senti_Value'] * df['Confi_Weight']

In [133]:
# Saving the final table

df.to_csv('feed_complete.csv')

In [134]:
# Final score based on Feed data

feed_score = df['Overall'].sum()
print(feed_score)

0.07692945281054789
